# spacy

- interface for data to be processed should be described (eg which columns are needed)
- display full df instead of head to have full featureset available
- added uv option for better package handling

In [1]:
import pandas as pd
from utils import preprocess_data, get_matches, select_matches

In [2]:
import os
from pathlib import Path
import duckdb as ddb
from pandas_plots import tbl, pls, hlp

hlp.show_package_version()
os.environ['THEME']='dark'

dir_db=Path("C://temp") if hlp.get_os(hlp.OperatingSystem.WINDOWS) else Path(os.path.expanduser("~/tmp"))

# ! use either clin or fake data
file_db_clin = dir_db/'2025-04-07_data_clin.db'

if not file_db_clin.exists():
    raise FileNotFoundError(f"File not found: {file_db_clin}")

🐍 3.11.7 | 📦 pandas: 2.2.3 | 📦 numpy: 1.26.4 | 📦 duckdb: 1.2.2 | 📦 pandas-plots: 0.12.23 | ❌ connection_helper: Not found


In [3]:
con = ddb.connect()
_=con.execute("PRAGMA disable_progress_bar;")
_=con.execute(f"ATTACH DATABASE '{file_db_clin}' as clin; set schema 'clin';")

In [4]:
#sub_data = pd.read_csv("C:/Substanzen/substances.csv", sep = ";", encoding="latin1")
#sub_data["Bezeichnung"] = sub_data["Bezeichnung"].replace({pd.NA: 'NA', '': 'NA'})

# * use local test data
# sub_data = pd.read_csv("C:/Substanzen/Test_Daten.csv", sep = ";", encoding="utf-8")

# * use real data
sub_data = con.sql("select distinct Bezeichnung from Substanz").to_df()
display(sub_data)

,Bezeichnung
0,Carboplatin
1,Trastuzumab
2,Hydroxycarbamid
3,Bicalutamid
4,Bacillus-Calmette-Guerin
...,...
4175,Sixantone
4176,Irinotecan peglip.
4177,Folsäure p.o.
4178,PAZOPANIB


In [5]:
URL_LINK = "https://gitlab.opencode.de/robert-koch-institut/zentrum-fuer-krebsregisterdaten/cancerdata-references/-/raw/main/data/v2/Klassifikationen/substanz.csv?ref_type=heads"
reference_list = pd.read_csv(URL_LINK, sep=";")
reference_list.head()

,therapieart,substanz,code
0,HO,Abarelix,L02BX01
1,IM,Abatacept,L04AA24
2,ZS,Abemaciclib,L01EF03
3,IM,Abetimus,L04AA22
4,HO,Abirateron,L02BX03


In [6]:
col_with_substances = sub_data["Bezeichnung"]
col_with_ref_substances = reference_list["substanz"]

In [7]:
def create_service_variable(
    col_with_free_text: pd.Series,
    col_with_refs: pd.Series,
    threshold_parameter: int = 85,
    pattern_to_split: str = r"[/,;+]|\bund\b|\boder\b",
) -> pd.DataFrame:
    """applies all the function defined in the utils.py file

    Args:
        col_with_free_text (pd.Series): The column with text which should be scanned for substances
        col_with_refs (pd.Series): The column with substances that we want to search for in the text
        threshold_parameter (int, optional): Defines the accuracy, higher value means more accuracy.
        Defaults to 85.
        pattern_to_split (str, optional): Defines when more than one match is allowed
        Defaults to r"[/,;+]|\bund\b|\boder\b".

    Raises:
        ValueError: checks whether all IDs from input can be found in the output
        ValueError: checks whether the number of rows is the same in in- and output

    Returns:
        pd.DataFrame: processed df with original input text,
        matched substances and the corresponding accuracy score
    """
    preprocessed_data = preprocess_data(col_with_free_text)

    matches_df = get_matches(
        preprocessed_data, col_with_refs, threshold_parameter=threshold_parameter
    )

    selected_matches_df = select_matches(matches_df, pattern_to_split=pattern_to_split)

    if not preprocessed_data["ID"].isin(selected_matches_df["ID"]).all():
        raise ValueError("Not all IDs from input are in output")

    if len(preprocessed_data) != len(selected_matches_df):
        raise ValueError("Length of input and output differs")

    out_df = preprocessed_data.merge(selected_matches_df, on="ID", how="left")

    return out_df

In [8]:
substances_with_service_variable = create_service_variable(
        col_with_substances, col_with_ref_substances
    )
substances_with_service_variable.to_csv(".local/output_AG_daten.csv", sep=";", index=False)

In [9]:
substances_with_service_variable

,ID,Original,Preprocessed_text,input,match,matched_to,similarity
0,1,Carboplatin,Carboplatin,Carboplatin,Carboplatin,Carboplatin,100
1,2,Trastuzumab,Trastuzumab,Trastuzumab,Trastuzumab,Trastuzumab,100
2,3,Hydroxycarbamid,Hydroxycarbamid,Hydroxycarbamid,Hydroxycarbamid,Hydroxycarbamid,100
3,4,Bicalutamid,Bicalutamid,Bicalutamid,Bicalutamid,Bicalutamid,100
4,5,Bacillus-Calmette-Guerin,Bacillus-Calmette-Guerin,Bacillus-Calmette-Guerin,,,
...,...,...,...,...,...,...,...
4175,4176,Sixantone,Sixantone,Sixantone,,,
4176,4177,Irinotecan peglip.,Irinotecan peglip.,Irinotecan peglip.,Irinotecan,Irinotecan,100
4177,4178,Folsäure p.o.,Folsäure p.o.,Folsäure p.o.,Folsäure,Folinsäure,89
4178,4179,PAZOPANIB,PAZOPANIB,PAZOPANIB,PAZOPANIB,Pazopanib,100


In [12]:
matches_counter = (substances_with_service_variable['matched_to'].notna() & (substances_with_service_variable['matched_to'] != "")).sum()
total_rows = len(substances_with_service_variable)
proportion = matches_counter / total_rows

print(f"score = {proportion:.3f} (matches / total rows)")

score = 0.571 (matches / total rows)
